# 基于Transformers的命名实体识别

## Step1 导入相关包

In [1]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

## Step2 加载数据集

In [2]:
ner_datasets = load_dataset("peoples_daily_ner", cache_dir="./data")
ner_datasets

Using the latest cached version of the module from C:\Users\yuyao\.cache\huggingface\modules\datasets_modules\datasets\peoples_daily_ner\594461a1b34f61af9346123a420b9ea40f15c0e835562053bf025cef188477f5 (last modified on Sat Jun 10 10:58:53 2023) since it couldn't be found locally at peoples_daily_ner., or remotely on the Hugging Face Hub.
Found cached dataset peoples_daily_ner (c:/Users/yuyao/Desktop/transformers/09-token_classification/data/peoples_daily_ner/peoples_daily_ner/1.0.0/594461a1b34f61af9346123a420b9ea40f15c0e835562053bf025cef188477f5)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
ner_datasets["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [4]:
ner_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [5]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [7]:
tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True)   # 对于已经做好tokenize的数据，要指定is_split_into_words参数为True

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
res = tokenizer("interesting word")
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [9]:
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [10]:
# 借助word_ids 实现标签映射
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples

In [11]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Loading cached processed dataset at c:\Users\yuyao\Desktop\transformers\09-token_classification\data\peoples_daily_ner\peoples_daily_ner\1.0.0\594461a1b34f61af9346123a420b9ea40f15c0e835562053bf025cef188477f5\cache-4a4f4b6e9ab5eb0d.arrow
Loading cached processed dataset at c:\Users\yuyao\Desktop\transformers\09-token_classification\data\peoples_daily_ner\peoples_daily_ner\1.0.0\594461a1b34f61af9346123a420b9ea40f15c0e835562053bf025cef188477f5\cache-ac83ba082615fd83.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [12]:
print(tokenized_datasets["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## Step4 创建模型

In [21]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized f

In [14]:
model.config.num_labels

7

## Step5 创建评估函数

In [15]:
seqeval = evaluate.load("seqeval")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [20]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    return {
        "f1": result["overall_f1"]
    }
    

## Step6 配置训练参数

In [23]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3
)

## Step7 创建训练器

In [24]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

## Step8 模型训练

In [25]:
trainer.train()

c:\Users\yuyao\miniconda3\envs\transformers\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/981 [00:00<?, ?it/s]

{'loss': 0.2358, 'learning_rate': 4.745158002038736e-05, 'epoch': 0.15}
{'loss': 0.0444, 'learning_rate': 4.490316004077472e-05, 'epoch': 0.31}
{'loss': 0.0364, 'learning_rate': 4.235474006116208e-05, 'epoch': 0.46}
{'loss': 0.0331, 'learning_rate': 3.980632008154944e-05, 'epoch': 0.61}
{'loss': 0.0301, 'learning_rate': 3.72579001019368e-05, 'epoch': 0.76}
{'loss': 0.0272, 'learning_rate': 3.4709480122324164e-05, 'epoch': 0.92}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.019287142902612686, 'eval_f1': 0.9381668946648427, 'eval_runtime': 2.93, 'eval_samples_per_second': 791.476, 'eval_steps_per_second': 6.485, 'epoch': 1.0}
{'loss': 0.0346, 'learning_rate': 3.2161060142711516e-05, 'epoch': 1.07}
{'loss': 0.0139, 'learning_rate': 2.9612640163098882e-05, 'epoch': 1.22}
{'loss': 0.015, 'learning_rate': 2.7064220183486238e-05, 'epoch': 1.38}
{'loss': 0.0126, 'learning_rate': 2.45158002038736e-05, 'epoch': 1.53}
{'loss': 0.0148, 'learning_rate': 2.196738022426096e-05, 'epoch': 1.68}
{'loss': 0.0133, 'learning_rate': 1.9418960244648318e-05, 'epoch': 1.83}
{'loss': 0.0133, 'learning_rate': 1.6870540265035677e-05, 'epoch': 1.99}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.016634825617074966, 'eval_f1': 0.9501706484641638, 'eval_runtime': 2.9165, 'eval_samples_per_second': 795.118, 'eval_steps_per_second': 6.515, 'epoch': 2.0}
{'loss': 0.0077, 'learning_rate': 1.4322120285423038e-05, 'epoch': 2.14}
{'loss': 0.0068, 'learning_rate': 1.1773700305810397e-05, 'epoch': 2.29}
{'loss': 0.007, 'learning_rate': 9.225280326197758e-06, 'epoch': 2.45}
{'loss': 0.0065, 'learning_rate': 6.676860346585118e-06, 'epoch': 2.6}
{'loss': 0.0052, 'learning_rate': 4.128440366972477e-06, 'epoch': 2.75}
{'loss': 0.007, 'learning_rate': 1.580020387359837e-06, 'epoch': 2.91}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.01770414039492607, 'eval_f1': 0.9536061311071575, 'eval_runtime': 2.9431, 'eval_samples_per_second': 787.934, 'eval_steps_per_second': 6.456, 'epoch': 3.0}
{'train_runtime': 175.2274, 'train_samples_per_second': 357.222, 'train_steps_per_second': 5.598, 'train_loss': 0.028998792156288505, 'epoch': 3.0}


TrainOutput(global_step=981, training_loss=0.028998792156288505, metrics={'train_runtime': 175.2274, 'train_samples_per_second': 357.222, 'train_steps_per_second': 5.598, 'train_loss': 0.028998792156288505, 'epoch': 3.0})

In [26]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.02416740171611309,
 'eval_f1': 0.9487282463186077,
 'eval_runtime': 5.9975,
 'eval_samples_per_second': 773.157,
 'eval_steps_per_second': 6.169,
 'epoch': 3.0}

## Step9 模型预测

In [27]:
from transformers import pipeline

In [33]:
# 使用pipeline进行推理，要指定id2label
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  

In [36]:
# 如果模型是基于GPU训练的，那么推理时要指定device
# 对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")

In [40]:
res = ner_pipe("小明在北京上班")
res

[{'entity_group': 'PER',
  'score': 0.74938077,
  'word': '小 明',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': 0.9992614,
  'word': '北 京',
  'start': 3,
  'end': 5}]

In [41]:
# 根据start和end取实际的结果
ner_result = {}
x = "小明在北京上班"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

ner_result

{'PER': ['小明'], 'LOC': ['北京']}